### Note this code takes 12 hours to run and extract all the data.
All of the code is the raw data cleaning code

In [1]:
#!pip install python-chess
# /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
#!brew install stockfish
#!pip install stockfish
# https://stockfishchess.org/download/

import chess.pgn
import re
import pandas as pd
import statistics 
import numpy as np

In [2]:
import chess.pgn
import io
import pandas as pd

# Function to extract clock time from a comment
def extract_clock(comment):
    if "[%clk" in comment:
        start = comment.find("[%clk") + 5
        end = comment.find("]", start)
        return comment[start:end]
    return "N/A"  # Return "N/A" if no clock time is found

# Initialize a list to hold the game data and a counter for the games
games = []
n = 0  # Counter for every game read
games_count = 0  # Counter for processed games

# Open the PGN file
with open('/Users/macuser1/Desktop/third year /st312/data/lichess_db_standard_rated_2023-12.pgn') as pgn_file:
    while True:
        # Increment the counter for each game read
        n += 1

        # Break if the desired number of games is processed
        #### kind of useless to filter by every 1000th game tbh, since still need to read the pgn file
        
        if games_count == 10000:   
            break

        # Read the next game from the file
        game = chess.pgn.read_game(pgn_file)
        
        # Break if no more games are found
        if game is None:
            break
            
        ## Need to do this cause sometimes the way they document each year is differrent, 
        ## Only doing this to test generalisability
        try:

            white_elo = int(game.headers.get("WhiteElo", "0"))
            black_elo = int(game.headers.get("BlackElo", "0"))


            if white_elo >= 2000 and black_elo >= 2000:
                moves_list = []
                clocks_list = []

                # Iterate through all nodes in the mainline to extract moves and clock times
                for node in game.mainline():
                    move = node.move
                    comment = node.comment
                    clock_time = extract_clock(comment)

                    moves_list.append(move.uci())  # Store move in UCI format
                    clocks_list.append(clock_time)  # Store clock time

                # Prepare game info with moves and clock times
                game_info = [
                    game.headers["Event"],
                    game.headers["Site"],
                    game.headers["Date"],
                    game.headers["Round"],
                    game.headers["White"],
                    game.headers["Black"],
                    game.headers["Result"],
                    game.headers["UTCDate"],
                    game.headers["UTCTime"],
                    game.headers["WhiteElo"],
                    game.headers["BlackElo"],
                    game.headers.get("WhiteRatingDiff", "N/A"),
                    game.headers.get("BlackRatingDiff", "N/A"),
                    game.headers.get("WhiteTitle", "N/A"),
                    game.headers.get("BlackTitle", "N/A"),
                    game.headers["ECO"],
                    game.headers["Opening"],
                    game.headers["TimeControl"],
                    game.headers["Termination"],
                    game.mainline_moves(),  # Add moves in UCI format
                    moves_list,  # Add moves list (also in UCI format but extracted alongside clock times)
                    clocks_list  # Add clock times list
                ]

                games.append(game_info)
                games_count += 1
                
                
                
                
        except Exception as e:
            # If any error occurs, skip this game and continue with the next
            continue

# defining the column names 
column_names = ["Event", "Site", "Date", "Round", "White", "Black", "Result", "UTCDate", "UTCTime", 
                "WhiteElo", "BlackElo", "WhiteRatingDiff", "BlackRatingDiff", "WhiteTitle", "BlackTitle", 
                "ECO", "Opening", "TimeControl", "Termination", "Moves", "Moves_extracted", "Clock_extracted"]

# Convert to df
games_df = pd.DataFrame(games, columns=column_names)


In [3]:
games_count

10000

In [4]:
len(games)

10000

In [5]:
column_names = ["Event", "Site", "Date", "Round", "White", "Black", "Result", "UTCDate", "UTCTime", 
                "WhiteElo", "BlackElo", "WhiteRatingDiff", "BlackRatingDiff", "WhiteTitle", "BlackTitle", 
                "ECO", "Opening", "TimeControl", "Termination", "Moves", "Moves_extracted", "Clock_extracted"]


games_df = pd.DataFrame(games, columns=column_names)

In [6]:
games_df.tail()

,Event,Site,Date,Round,White,Black,Result,UTCDate,UTCTime,WhiteElo,...,BlackRatingDiff,WhiteTitle,BlackTitle,ECO,Opening,TimeControl,Termination,Moves,Moves_extracted,Clock_extracted
9995,Rated Bullet game,https://lichess.org/1Fwc5ZHC,2023.12.01,-,Turukmakkto,Mustapha_mouri,1-0,2023.12.01,00:43:56,2217,...,-3,N/A,N/A,A00,Hungarian Opening,30+0,Normal,1. g3 { [%clk 0:00:30] } 1... e5 { [%clk 0:00:...,"[g2g3, e7e5, d2d4, d7d5, d4e5, b8c6, g1f3, g8e...","[ 0:00:30, 0:00:30, 0:00:30, 0:00:30, 0:00..."
9996,Rated Blitz game,https://lichess.org/1yJPtD8Q,2023.12.01,-,Abel333,wincero,0-1,2023.12.01,00:42:31,2190,...,+7,N/A,N/A,D15,Slav Defense: Three Knights Variation,180+0,Normal,1. d4 { [%clk 0:03:00] } 1... c6 { [%clk 0:03:...,"[d2d4, c7c6, c2c4, d7d5, b1c3, g8f6, g1f3, c8g...","[ 0:03:00, 0:03:00, 0:02:58, 0:02:59, 0:02..."
9997,Rated Blitz game,https://lichess.org/5JC73IiQ,2023.12.01,-,vinrabino,Gambit203,1-0,2023.12.01,00:42:31,2113,...,-5,N/A,N/A,A48,London System,180+0,Time forfeit,1. d4 { [%clk 0:03:00] } 1... Nf6 { [%clk 0:03...,"[d2d4, g8f6, g1f3, g7g6, c1f4, f8g7, e2e3, e8g...","[ 0:03:00, 0:03:00, 0:03:00, 0:02:58, 0:03..."
9998,Rated Blitz game,https://lichess.org/1WpAswqT,2023.12.01,-,pjdoc,sri_santiko,1-0,2023.12.01,00:42:31,2175,...,-6,N/A,N/A,B01,Scandinavian Defense: Mieses-Kotroc Variation,300+0,Normal,1. e4 { [%clk 0:05:00] } 1... d5 { [%clk 0:05:...,"[e2e4, d7d5, e4d5, d8d5, g1f3, g8f6, d2d4, c8g...","[ 0:05:00, 0:05:00, 0:04:58, 0:04:58, 0:04..."
9999,Rated Bullet game,https://lichess.org/4GMV3wZI,2023.12.01,-,wpkq,Lawzi,1-0,2023.12.01,00:42:31,2171,...,-5,N/A,N/A,B10,Caro-Kann Defense: Two Knights Attack,60+0,Time forfeit,1. e4 { [%clk 0:01:00] } 1... c6 { [%clk 0:01:...,"[e2e4, c7c6, g1f3, d7d5, b1c3, e7e6, e4d5, e6d...","[ 0:01:00, 0:01:00, 0:01:00, 0:01:00, 0:01..."


In [7]:
games_df.shape

(10000, 22)

## Removing duplicate players 

In [8]:
# Combine the 'White' and 'Black' columns 
all_players = pd.concat([games_df['White'], games_df['Black']]).unique()

# Create a dictionary to keep track of whether a player has been added
player_added = {player: False for player in all_players}

# Check if the player has been added already
def check_player(row):
    
    keep_row = False
    if not player_added[row['White']]:
        player_added[row['White']] = True
        keep_row = True
    if not player_added[row['Black']]:
        player_added[row['Black']] = True
        keep_row = True
        
    return keep_row

games_df = games_df[games_df.apply(check_player, axis=1)]

"\n\n# Combine the 'White' and 'Black' columns into a single Series of player names\nall_players = pd.concat([games_df['White'], games_df['Black']]).unique()\n\n# Create a dictionary to keep track of whether a player has been added\nplayer_added = {player: False for player in all_players}\n\n# Function to check if the player has been added already\ndef check_player(row):\n    \n    keep_row = False\n    if not player_added[row['White']]:\n        player_added[row['White']] = True\n        keep_row = True\n    if not player_added[row['Black']]:\n        player_added[row['Black']] = True\n        keep_row = True\n        \n    return keep_row\n\n# Apply the function to filter the DataFrame\ngames_df = games_df[games_df.apply(check_player, axis=1)]\n\n"

In [9]:
games_df.shape

(10000, 22)

## Adding fen to df_games

So this part just adds a new column called FEN, which is just a notation of chess positions, so the FEN column is a list which has the same length as the moves column. 

In [10]:
def san_to_fen(mainline):
    # Create a new chess board
    board = chess.Board()
    
    # Initialize a list to store FENs
    fen_sequence = []

    # Iterate over the moves in the mainline
    for move in mainline:
        board.push(move)
        fen_sequence.append(board.fen())

    return fen_sequence

In [11]:
games_df['FEN'] = games_df['Moves'].apply(san_to_fen)


In [12]:
games_df.head()

,Event,Site,Date,Round,White,Black,Result,UTCDate,UTCTime,WhiteElo,...,WhiteTitle,BlackTitle,ECO,Opening,TimeControl,Termination,Moves,Moves_extracted,Clock_extracted,FEN
0,Rated Bullet game,https://lichess.org/Tj2MPysr,2023.12.01,-,Pumpernickel73,Lupus8173,0-1,2023.12.01,00:00:22,2062,...,N/A,N/A,B40,"Sicilian Defense: French Variation, Westerinen...",120+1,Time forfeit,1. e4 { [%clk 0:02:00] } 1... c5 { [%clk 0:02:...,"[e2e4, c7c5, g1f3, e7e6, b2b3, b8c6, c1b2, d7d...","[ 0:02:00, 0:02:00, 0:01:59, 0:02:00, 0:01...",[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...
1,Rated Bullet game,https://lichess.org/j4rXgfqP,2023.12.01,-,CarlosCarneiro,Pilchowitz,1-0,2023.12.01,00:00:22,2167,...,NM,N/A,D15,Slav Defense: Geller Gambit,120+1,Normal,1. d4 { [%clk 0:02:00] } 1... d5 { [%clk 0:02:...,"[d2d4, d7d5, g1f3, g8f6, c2c4, c7c6, b1c3, d5c...","[ 0:02:00, 0:02:00, 0:01:59, 0:02:00, 0:01...",[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...
2,Rated Bullet game,https://lichess.org/zhrwxixV,2023.12.01,-,mtm_course,wangzihang,0-1,2023.12.01,00:00:22,2381,...,N/A,N/A,C00,French Defense: Two Knights Variation,120+1,Normal,1. e4 { [%clk 0:02:00] } 1... e6 { [%clk 0:02:...,"[e2e4, e7e6, g1f3, d7d5, b1c3, d5d4, c3e2, c7c...","[ 0:02:00, 0:02:00, 0:02:00, 0:02:00, 0:02...",[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...
3,Rated Bullet game,https://lichess.org/BOtSsf59,2023.12.01,-,Zamzurigenjur,Bulletaria,0-1,2023.12.01,00:00:23,2193,...,N/A,N/A,A40,Horwitz Defense,60+0,Normal,1. d4 { [%clk 0:01:00] } 1... e6 { [%clk 0:01:...,"[d2d4, e7e6, b1c3, d7d5, g1f3, f8e7, c1g5, g8f...","[ 0:01:00, 0:01:00, 0:01:00, 0:01:00, 0:01...",[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...
4,Rated Bullet game,https://lichess.org/vxbfuE3b,2023.12.01,-,LudwigJam,ahmadalfraa,1-0,2023.12.01,00:00:23,2219,...,N/A,N/A,B01,Scandinavian Defense,60+0,Normal,1. e4 { [%clk 0:01:00] } 1... d5 { [%clk 0:01:...,"[e2e4, d7d5, b1c3, d5e4, c3e4, f7f5, e4c3, e7e...","[ 0:01:00, 0:01:00, 0:01:00, 0:00:59, 0:01...",[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...


## Adding sd and score to the df

So now that we have FEN, we use it as a common column in order to add the sd to all the positions,


so we add the new columns:

1. sd - list (taken from the gpickle)
2. average sd - number
3. score - list (taken from the gpickle)

The gpickle we obtained was from messaging the author of the paper directly

In [13]:
import pickle

#with open('copy_690k_30_sd.gpickle', 'rb') as file:
#    g = pickle.load(file)


with open('Full_Merge_500.gpickle', 'rb') as file:
    g = pickle.load(file)
    

In [14]:
len(g.nodes())

904563

In [15]:
# create a mapping of FEN to standard deviation
fen_to_sd = {fen: data.get('sd') for fen, data in g.nodes(data=True)}
fen_to_score  = {fen: data.get('score') for fen, data in g.nodes(data=True)}

# function to map FEN sequences to sd
def get_sd_for_fen_sequence(fen_sequence):
    return [fen_to_sd.get(fen) for fen in fen_sequence]

def get_score_for_fen_sequence(fen_sequence):
    return [fen_to_score.get(fen) for fen in fen_sequence]

def mean_with_none_handling(sd_list):
    # Filter out None values
    filtered_list = [x for x in sd_list if x is not None]
    return statistics.mean(filtered_list) if filtered_list else None



# applying function to each row in the df
games_df['sd'] = games_df['FEN'].apply(get_sd_for_fen_sequence)
games_df["average_sd"] = games_df["sd"].apply(mean_with_none_handling)
games_df['score'] = games_df['FEN'].apply(get_score_for_fen_sequence)


# to change results to 0,1,2
games_df['Result'] = games_df['Result'].map({'1-0': 1, '0-1': 0, '1/2-1/2': 2})


In [17]:
### Confirming that each node (aka chess position) in the gpickle occured at least 500 times 
##
lowest_count = min(data.get('count') for _, data in g.nodes(data=True) if data.get('count') is not None)
lowest_count

500

In [18]:
## To see how many is not None, ie per game how many moves have sd

l= []
how_many_moves_have_sd = []
for i in range(len(games_df) - 1):
    
    game = games_df.iloc[i]["sd"][:]
    count = 0
    count2 = 0
    
    for ii in game:
        if ii == None:
            count += 1
        else:
            count2+= 1
            
    if len(game) != 0:
        how_many_moves_have_sd.append(count2)
        l.append(count/ len(game))

    
    
print("Percentage of game that has sd: ", 1 - (sum(l) / len(l))) # is amount 
print("average moves that have sd: ", sum(how_many_moves_have_sd) / len(how_many_moves_have_sd))

Percentage of game that has sd:  0.14919148763725432
average moves that have sd:  9.238071421426428


In [19]:
len(games_df)

10000

In [20]:
games_df = games_df.iloc[:100000]

games_df = games_df.drop(['Site', 'Round', 'White', "Black",'UTCTime', 'BlackRatingDiff', 
                          'WhiteTitle', 'BlackTitle', 'Opening', 'FEN','Moves'], axis=1)




In [21]:
games_df.tail()

,Event,Date,Result,UTCDate,WhiteElo,BlackElo,WhiteRatingDiff,ECO,TimeControl,Termination,Moves_extracted,Clock_extracted,sd,average_sd,score
9995,Rated Bullet game,2023.12.01,1,2023.12.01,2217,2049,+3,A00,30+0,Normal,"[g2g3, e7e5, d2d4, d7d5, d4e5, b8c6, g1f3, g8e...","[ 0:00:30, 0:00:30, 0:00:30, 0:00:30, 0:00...","[46.455581750167646, 36.40405360137375, 77.054...",56.400170,"[-131, 56, -119, -135, 58, -89, 33, None, None..."
9996,Rated Blitz game,2023.12.01,0,2023.12.01,2190,2158,-7,D15,180+0,Normal,"[d2d4, c7c6, c2c4, d7d5, b1c3, g8f6, g1f3, c8g...","[ 0:03:00, 0:03:00, 0:02:58, 0:02:59, 0:02...","[54.75477948677897, 32.56921105165252, 31.1945...",42.722403,"[0, -110, 15, -78, -37, -77, -14, -125, 12, -1..."
9997,Rated Blitz game,2023.12.01,1,2023.12.01,2113,2081,+5,A48,180+0,Time forfeit,"[d2d4, g8f6, g1f3, g7g6, c1f4, f8g7, e2e3, e8g...","[ 0:03:00, 0:03:00, 0:03:00, 0:02:58, 0:03...","[54.75477948677897, 29.757908204661298, 30.456...",31.737359,"[0, -71, 12, -77, 5, -62, -22, -87, -10, -66, ..."
9998,Rated Blitz game,2023.12.01,1,2023.12.01,2175,2200,+6,B01,300+0,Normal,"[e2e4, d7d5, e4d5, d8d5, g1f3, g8f6, d2d4, c8g...","[ 0:05:00, 0:05:00, 0:04:58, 0:04:58, 0:04...","[24.39086901598828, 96.03744698385567, 59.0352...",78.167103,"[51, -84, 36, -160, 135, -133, 44, -200, 99, -..."
9999,Rated Bullet game,2023.12.01,1,2023.12.01,2171,2153,+6,B10,60+0,Time forfeit,"[e2e4, c7c6, g1f3, d7d5, b1c3, e7e6, e4d5, e6d...","[ 0:01:00, 0:01:00, 0:01:00, 0:01:00, 0:01...","[24.39086901598828, 41.23404899832644, 42.9866...",37.709265,"[51, -110, 59, -81, -11, -102, -2, -33, -23, -..."


In [22]:
######################## import to csv for the first df
games_df.to_csv("results_temp/final_overnight_df1_2023_12.csv")

# For reserach question 1


## preparing the data

In [34]:
import pandas as pd

def clock_to_seconds(clock_str):
    """Convert clock string to total seconds."""
    
    if clock_str == 'N/A':
        return None
    h, m, s = map(int, clock_str.split(':'))
    return h * 3600 + m * 60 + s



structured_data = []
faulty_list = []

for index, game in games_df.iterrows():
    
    if game['TimeControl'] == "-":     #### Sometimes they didnt record the time 
        faulty_list.append(index)
        continue
    

    moves = game['Moves_extracted']
    clocks = game['Clock_extracted']
    game_sds = game['sd']
    game_scores = game['score']
    time_control = game['TimeControl']
    increment = int(time_control.split('+')[1])
    
    # list to hold data for the current game
    temp_structured_data = []
    negative_time_spent_found = False
    
    if len(moves) == 0:       ##### for the abandoned games 
        faulty_list.append(index)
        continue

    for i in range(len(moves)):
        move = moves[i]
        game_sd = game_sds[i]
        game_score = game_scores[i]
        clock = clocks[i] if i < len(clocks) else None
        curr_clock_seconds = clock_to_seconds(clock) if clock else None
        
        # Determine player (White or Black) based on move index
        player = 'White' if i % 2 == 0 else 'Black'

        # Calculate time spent considering the clock time from two moves prior
        if i > 1 and curr_clock_seconds is not None:
            prev_clock_seconds = clock_to_seconds(clocks[i-2])
            time_spent = prev_clock_seconds - curr_clock_seconds + increment
        else:
            time_spent = 0 

        # Check if time_spent is negative
        if time_spent < 0:
            negative_time_spent_found = True
            faulty_list.append(index)
            break  # if negative time found exit loop early

        # Add move data to temporary structured data
        temp_structured_data.append({
            'GameIndex': index,
            'Move': move,
            'Clock': clock,
            'TimeSpent': time_spent,
            'Player': player,
            'SD': game_sd,
            'Score': game_score,
            **{col: game[col] for col in games_df.columns if col not in ['Moves_extracted', 'Clock_extracted', 
                                                                         'sd', 'score', "FEN"]}
        })

    # Only add the game's data to structured_data if no negative time spent was found
    if not negative_time_spent_found:
        structured_data.extend(temp_structured_data)

structured_df = pd.DataFrame(structured_data)


In [35]:
## Check how many unique games are in the df
structured_df["GameIndex"].nunique()

9960

# Checking/Debugging stuff for the multi-level model

In [36]:
### to check min value for time spent 
print(min(structured_df["TimeSpent"]))


### Checking valid games and faulty ones

print(structured_df["GameIndex"].nunique())

print(len(faulty_list))

0
9960
40


In [37]:
len(faulty_list)


40

In [38]:
#### Used to debug/check issue with any certain chess game


def print_move_times(row_num):
    # Ensure row_num is within the bounds of games_df
    if row_num < 0 or row_num >= len(games_df):
        print("Row number is out of bounds.")
        return
    
    game = games_df.iloc[row_num]
    moves = game['Moves_extracted']
    clocks = game['Clock_extracted']
    time_control = game['TimeControl']
    
    
    if game['TimeControl'] == "-":
        return "time not recorded"
    
    increment = int(time_control.split('+')[1])
    
    
    print(f"TimeControl: {time_control}")
    print(f"Game at row {row_num}:")
    print(f"{'Move':<10} {'Player':<10} {'Prev Clock':<12} {'Curr Clock':<12} {'Time Spent':<10}")
    
    for i in range(len(moves)):
        move = moves[i]
        player = 'White' if i % 2 == 0 else 'Black'
        curr_clock = clocks[i] if i < len(clocks) else None
        
        if i > 1 and curr_clock is not None:
            prev_clock = clocks[i-2]
            curr_clock_seconds = clock_to_seconds(curr_clock)
            prev_clock_seconds = clock_to_seconds(prev_clock)
            time_spent = prev_clock_seconds - curr_clock_seconds + increment
        else:
            prev_clock = None
            time_spent = None
        
        print(f"{move:<10} {player:<10} {prev_clock if prev_clock else 'N/A':<12} {curr_clock if curr_clock else 'N/A':<12} {time_spent if time_spent else 'N/A':<10}")

# Example usage
print_move_times(321)  


TimeControl: 600+0
Game at row 321:
Move       Player     Prev Clock   Curr Clock   Time Spent
d2d4       White      N/A           0:10:00     N/A       
d7d5       Black      N/A           0:10:00     N/A       
c2c4       White       0:10:00      0:09:56     4         
b8c6       Black       0:10:00      0:09:58     2         
g1f3       White       0:09:56      0:09:52     4         
e7e6       Black       0:09:58      0:09:57     1         
b1c3       White       0:09:52      0:09:48     4         
g8f6       Black       0:09:57      0:09:56     1         
c1g5       White       0:09:48      0:09:46     2         
f8b4       Black       0:09:56      0:09:55     1         
e2e3       White       0:09:46      0:09:46     N/A       
h7h6       Black       0:09:55      0:09:53     2         
g5f6       White       0:09:46      0:09:41     5         
g7f6       Black       0:09:53      0:09:53     N/A       
d1c2       White       0:09:41      0:09:29     12        
a7a6       Black    

### Removing the rows without riskiness


In [39]:
### how many rows do not have riiskiness

structured_df['SD'].isna().sum() / len(structured_df)

0.8790985275915624

In [40]:
## removes rows with no SD

structured_df = structured_df.dropna(subset=['SD'])


In [41]:
structured_df.to_csv("results_temp/overnight_df2_2023_12.csv")

## this df smaller than df1 cause removed rows without risk

In [42]:
len(structured_df)

92014

In [43]:
structured_df.tail()

,GameIndex,Move,Clock,TimeSpent,Player,SD,Score,Event,Date,Result,UTCDate,WhiteElo,BlackElo,WhiteRatingDiff,ECO,TimeControl,Termination,average_sd
760997,9999,e4d5,0:00:59,1,White,16.263456,-2.0,Rated Bullet game,2023.12.01,1,2023.12.01,2171,2153,+6,B10,60+0,Time forfeit,37.709265
760998,9999,e6d5,0:00:57,2,Black,35.908626,-33.0,Rated Bullet game,2023.12.01,1,2023.12.01,2171,2153,+6,B10,60+0,Time forfeit,37.709265
760999,9999,d2d4,0:00:59,0,White,52.967019,-23.0,Rated Bullet game,2023.12.01,1,2023.12.01,2171,2153,+6,B10,60+0,Time forfeit,37.709265
761000,9999,g8f6,0:00:57,0,Black,67.457430,-80.0,Rated Bullet game,2023.12.01,1,2023.12.01,2171,2153,+6,B10,60+0,Time forfeit,37.709265
761001,9999,f1d3,0:00:59,0,White,5.524521,40.0,Rated Bullet game,2023.12.01,1,2023.12.01,2171,2153,+6,B10,60+0,Time forfeit,37.709265


# Splitting into game type like rapid, blitz, classsical

## For Multi level model


In [44]:
import pandas as pd
# Load the CSV file
# file_path = 
df = pd.read_csv(file_path)

# Filter out entries in the 'Event' column that are URLs
# This uses a simple string check. For more complex URLs, you might need regex.
df_filtered = df[~df['Event'].str.startswith(('http://', 'https://'), na=False)]

# Get unique values in the 'Event' column after filtering
unique_events = df_filtered['Event'].unique()

# Print unique non-URL values
print(unique_events)


['Rated Bullet game' 'Rated Rapid game' 'Rated Blitz game'
 'Rated Blitz tournament https://lichess.org/tournament/wuG2ZhHI'
 'Rated UltraBullet game' 'Rapid swiss https://lichess.org/swiss/7gtgXIu8'
 'Rated Blitz tournament https://lichess.org/tournament/DZI8gDuc'
 'Rated Blitz tournament https://lichess.org/tournament/UIdqsVev'
 'Rated Blitz tournament https://lichess.org/tournament/P38Dovm5'
 'Rated Bullet tournament https://lichess.org/tournament/EgZ157WC'
 'Rated Bullet tournament https://lichess.org/tournament/qhH1rnEj'
 'Rated Rapid tournament https://lichess.org/tournament/TQjuizi0'
 'Rated Classical game' 'Rapid swiss https://lichess.org/swiss/xfS7FAhs'
 'Rated Blitz tournament https://lichess.org/tournament/47hlkGPR'
 'Rapid swiss https://lichess.org/swiss/ko5lSXHw'
 'Rated Blitz tournament https://lichess.org/tournament/JR7UmBZx'
 'Blitz swiss https://lichess.org/swiss/4tLnHP3e'
 'Rated Blitz tournament https://lichess.org/tournament/OqnoiZi6'
 'Blitz swiss https://lichess.o

In [45]:
url_pattern = r'https?://[^\s]+'

df['Game_type'] = df['Event'].str.replace(url_pattern, '', regex=True)

In [46]:
words_pattern = r'\b(Rated|tournament|game)\b'

df['Game_type'] = df['Game_type'].str.replace(words_pattern, '', regex=True).str.strip()

df["Game_type"].unique()

array(['Bullet', 'Rapid', 'Blitz', 'UltraBullet', 'Rapid swiss',
       'Classical', 'Blitz swiss'], dtype=object)

In [47]:
# output_file_path =

# saving the csv
df.to_csv(output_file_path, index=False)


## Data cleaning for the multinomial logistic model


In [48]:
import pandas as pd

file_path = 
df = pd.read_csv(file_path)

# filtering out entries in the 'Event' column that are URLs
df_filtered = df[~df['Event'].str.startswith(('http://', 'https://'), na=False)]

# looking at the events that are contained in our dataset
unique_events = df_filtered['Event'].unique()

print(unique_events)

['Rated Bullet game' 'Rated Rapid game' 'Rated Blitz game'
 'Rated Blitz tournament https://lichess.org/tournament/wuG2ZhHI'
 'Rated UltraBullet game' 'Rated Correspondence game'
 'Rapid swiss https://lichess.org/swiss/7gtgXIu8'
 'Rated Blitz tournament https://lichess.org/tournament/DZI8gDuc'
 'Rated Blitz tournament https://lichess.org/tournament/UIdqsVev'
 'Rated Blitz tournament https://lichess.org/tournament/P38Dovm5'
 'Rated Bullet tournament https://lichess.org/tournament/EgZ157WC'
 'Rated Bullet tournament https://lichess.org/tournament/qhH1rnEj'
 'Rated Rapid tournament https://lichess.org/tournament/TQjuizi0'
 'Rated Classical game' 'Rapid swiss https://lichess.org/swiss/xfS7FAhs'
 'Rated Blitz tournament https://lichess.org/tournament/47hlkGPR'
 'Rapid swiss https://lichess.org/swiss/ko5lSXHw'
 'Rated Blitz tournament https://lichess.org/tournament/JR7UmBZx'
 'Blitz swiss https://lichess.org/swiss/4tLnHP3e'
 'Rated Blitz tournament https://lichess.org/tournament/OqnoiZi6'
 '

In [49]:

# Regular expression pattern to match URLs
url_pattern = r'https?://[^\s]+'

# Remove URLs and assign the result to a new column
df['Game_type'] = df['Event'].str.replace(url_pattern, '', regex=True)

df["Game_type"].unique()

array(['Rated Bullet game', 'Rated Rapid game', 'Rated Blitz game',
       'Rated Blitz tournament ', 'Rated UltraBullet game',
       'Rated Correspondence game', 'Rapid swiss ',
       'Rated Bullet tournament ', 'Rated Rapid tournament ',
       'Rated Classical game', 'Blitz swiss '], dtype=object)

In [50]:
# Regular expression pattern to match 'Rated', 'tournament', and 'game'
words_pattern = r'\b(Rated|tournament|game)\b'

# removing 'Rated', 'tournament', 'game' and assign the result back to the column
df['Game_type'] = df['Game_type'].str.replace(words_pattern, '', regex=True).str.strip()

# Extracting to find the game types that are in the dataset
df["Game_type"].unique()

array(['Bullet', 'Rapid', 'Blitz', 'UltraBullet', 'Correspondence',
       'Rapid swiss', 'Classical', 'Blitz swiss'], dtype=object)

In [51]:
# Specifying path of where to save the csv file
#output_file_path = 

# Saving the df as a csv file
#df.to_csv(output_file_path, index=False)


# df.head()


,Unnamed: 0,Event,Date,Result,UTCDate,WhiteElo,BlackElo,WhiteRatingDiff,ECO,TimeControl,Termination,Moves_extracted,Clock_extracted,sd,average_sd,score,Game_type
0,0,Rated Bullet game,2023.12.01,0,2023.12.01,2062,2113,-4.0,B40,120+1,Time forfeit,"['e2e4', 'c7c5', 'g1f3', 'e7e6', 'b2b3', 'b8c6...","[' 0:02:00', ' 0:02:00', ' 0:01:59', ' 0:02:00...","[24.39086901598828, 50.16194351128317, 27.5448...",25.221484,"[51, -106, 52, -129, -14, -74, 24, -60, 12, -5...",Bullet
1,1,Rated Bullet game,2023.12.01,1,2023.12.01,2167,2149,7.0,D15,120+1,Normal,"['d2d4', 'd7d5', 'g1f3', 'g8f6', 'c2c4', 'c7c6...","[' 0:02:00', ' 0:02:00', ' 0:01:59', ' 0:02:00...","[54.75477948677897, 34.15672810116084, 29.0607...",46.570704,"[0, -25, 17, -49, 12, -89, 25, -70, -59, -27, ...",Bullet
2,2,Rated Bullet game,2023.12.01,0,2023.12.01,2381,2478,-4.0,C00,120+1,Normal,"['e2e4', 'e7e6', 'g1f3', 'd7d5', 'b1c3', 'd5d4...","[' 0:02:00', ' 0:02:00', ' 0:02:00', ' 0:02:00...","[24.39086901598828, 52.32434288203594, 56.8426...",60.996020,"[51, -112, 4, -50, -1, -58, 11, -53, 6, -140, ...",Bullet
3,3,Rated Bullet game,2023.12.01,0,2023.12.01,2193,2171,-6.0,A40,60+0,Normal,"['d2d4', 'e7e6', 'b1c3', 'd7d5', 'g1f3', 'f8e7...","[' 0:01:00', ' 0:01:00', ' 0:01:00', ' 0:01:00...","[54.75477948677897, 55.34030404468891, 42.3223...",70.752660,"[0, -90, 33, -84, -23, -77, -411, -45, -136, -...",Bullet
4,4,Rated Bullet game,2023.12.01,1,2023.12.01,2219,2126,4.0,B01,60+0,Normal,"['e2e4', 'd7d5', 'b1c3', 'd5e4', 'c3e4', 'f7f5...","[' 0:01:00', ' 0:01:00', ' 0:01:00', ' 0:00:59...","[24.39086901598828, 96.03744698385567, 69.7629...",84.227783,"[51, -84, -97, -50, -5, -120, 63, -131, 18, No...",Bullet


# Correcting the average sd of black and white

In [52]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [53]:
# file_path = 
df  = pd.read_csv(file_path)

df.head()

,Unnamed: 0,Event,Date,Result,UTCDate,WhiteElo,BlackElo,WhiteRatingDiff,ECO,TimeControl,Termination,Moves_extracted,Clock_extracted,sd,average_sd,score,Game_type
0,0,Rated Bullet game,2023.12.01,0,2023.12.01,2062,2113,-4.0,B40,120+1,Time forfeit,"['e2e4', 'c7c5', 'g1f3', 'e7e6', 'b2b3', 'b8c6...","[' 0:02:00', ' 0:02:00', ' 0:01:59', ' 0:02:00...","[24.39086901598828, 50.16194351128317, 27.5448...",25.221484,"[51, -106, 52, -129, -14, -74, 24, -60, 12, -5...",Bullet
1,1,Rated Bullet game,2023.12.01,1,2023.12.01,2167,2149,7.0,D15,120+1,Normal,"['d2d4', 'd7d5', 'g1f3', 'g8f6', 'c2c4', 'c7c6...","[' 0:02:00', ' 0:02:00', ' 0:01:59', ' 0:02:00...","[54.75477948677897, 34.15672810116084, 29.0607...",46.570704,"[0, -25, 17, -49, 12, -89, 25, -70, -59, -27, ...",Bullet
2,2,Rated Bullet game,2023.12.01,0,2023.12.01,2381,2478,-4.0,C00,120+1,Normal,"['e2e4', 'e7e6', 'g1f3', 'd7d5', 'b1c3', 'd5d4...","[' 0:02:00', ' 0:02:00', ' 0:02:00', ' 0:02:00...","[24.39086901598828, 52.32434288203594, 56.8426...",60.996020,"[51, -112, 4, -50, -1, -58, 11, -53, 6, -140, ...",Bullet
3,3,Rated Bullet game,2023.12.01,0,2023.12.01,2193,2171,-6.0,A40,60+0,Normal,"['d2d4', 'e7e6', 'b1c3', 'd7d5', 'g1f3', 'f8e7...","[' 0:01:00', ' 0:01:00', ' 0:01:00', ' 0:01:00...","[54.75477948677897, 55.34030404468891, 42.3223...",70.752660,"[0, -90, 33, -84, -23, -77, -411, -45, -136, -...",Bullet
4,4,Rated Bullet game,2023.12.01,1,2023.12.01,2219,2126,4.0,B01,60+0,Normal,"['e2e4', 'd7d5', 'b1c3', 'd5e4', 'c3e4', 'f7f5...","[' 0:01:00', ' 0:01:00', ' 0:01:00', ' 0:00:59...","[24.39086901598828, 96.03744698385567, 69.7629...",84.227783,"[51, -84, -97, -50, -5, -120, 63, -131, 18, No...",Bullet


In [54]:
def safe_literal_eval(sd_str):
    try:
        return literal_eval(sd_str)
    except (ValueError, SyntaxError):
        return sd_str

Funtion to # Calculate average standard deviation of each game
def calculate_average_sds(sd):
    if isinstance(sd, str):
        sd = safe_literal_eval(sd)
    
    # Convert None values to np.nan and filter the list
    sd_list = [np.nan if x is None else x for x in sd]
    
    # Check if the list is empty or contains only np.nan after conversion
    if not sd_list or all(np.isnan(x) for x in sd_list):
        return np.nan, np.nan
    
    # Split sd_list into white and black moves
    white_sd = sd_list[::2]  # Every second element starting from 0 (white)
    black_sd = sd_list[1::2]  # Every second element starting from 1 (black)
    
    
    # Calculate average if both white and black have 5 values
    if len(white_sd) < 5 or len(black_sd) < 5:
        return np.nan, np.nan
    
    # Calculate the mean, np.nanmean already returns np.nan for empty lists
    average_white_sd = np.nanmean(white_sd)
    average_black_sd = np.nanmean(black_sd)
    
    return average_white_sd, average_black_sd


# Apply the function to each row and create new columns for the averages
df[['average_white_sd', 'average_black_sd']] = pd.DataFrame(df['sd'].apply(calculate_average_sds).tolist(), index=df.index)

# drop the 'average_sd' column 
if 'average_sd' in df.columns:
    df.drop('average_sd', axis=1, inplace=True)


In [55]:


nan_count_white = df['average_white_sd'].isna().sum()
nan_count_black = df['average_black_sd'].isna().sum()

print(f"Number of NaN in 'average_white_sd': {nan_count_white}")
print(f"Number of NaN in 'average_black_sd': {nan_count_black}")



Number of NaN in 'average_white_sd': 44
Number of NaN in 'average_black_sd': 44


In [56]:
# Drop rows where either 'average_white_sd' or 'average_black_sd' is NaN
df_cleaned = df.dropna(subset=['average_white_sd', 'average_black_sd']).copy()

# Explicitly adding the 'sd_difference' column 
df_cleaned.loc[:, 'sd_difference'] = df_cleaned['average_white_sd'] - df_cleaned['average_black_sd']


df_cleaned.head()

,Unnamed: 0,Event,Date,Result,UTCDate,WhiteElo,BlackElo,WhiteRatingDiff,ECO,TimeControl,Termination,Moves_extracted,Clock_extracted,sd,score,Game_type,average_white_sd,average_black_sd,sd_difference
0,0,Rated Bullet game,2023.12.01,0,2023.12.01,2062,2113,-4.0,B40,120+1,Time forfeit,"['e2e4', 'c7c5', 'g1f3', 'e7e6', 'b2b3', 'b8c6...","[' 0:02:00', ' 0:02:00', ' 0:01:59', ' 0:02:00...","[24.39086901598828, 50.16194351128317, 27.5448...","[51, -106, 52, -129, -14, -74, 24, -60, 12, -5...",Bullet,25.018949,25.424020,-0.405070
1,1,Rated Bullet game,2023.12.01,1,2023.12.01,2167,2149,7.0,D15,120+1,Normal,"['d2d4', 'd7d5', 'g1f3', 'g8f6', 'c2c4', 'c7c6...","[' 0:02:00', ' 0:02:00', ' 0:01:59', ' 0:02:00...","[54.75477948677897, 34.15672810116084, 29.0607...","[0, -25, 17, -49, 12, -89, 25, -70, -59, -27, ...",Bullet,48.817574,44.698313,4.119261
2,2,Rated Bullet game,2023.12.01,0,2023.12.01,2381,2478,-4.0,C00,120+1,Normal,"['e2e4', 'e7e6', 'g1f3', 'd7d5', 'b1c3', 'd5d4...","[' 0:02:00', ' 0:02:00', ' 0:02:00', ' 0:02:00...","[24.39086901598828, 52.32434288203594, 56.8426...","[51, -112, 4, -50, -1, -58, 11, -53, 6, -140, ...",Bullet,56.402677,66.508031,-10.105355
3,3,Rated Bullet game,2023.12.01,0,2023.12.01,2193,2171,-6.0,A40,60+0,Normal,"['d2d4', 'e7e6', 'b1c3', 'd7d5', 'g1f3', 'f8e7...","[' 0:01:00', ' 0:01:00', ' 0:01:00', ' 0:01:00...","[54.75477948677897, 55.34030404468891, 42.3223...","[0, -90, 33, -84, -23, -77, -411, -45, -136, -...",Bullet,52.270847,84.614019,-32.343171
4,4,Rated Bullet game,2023.12.01,1,2023.12.01,2219,2126,4.0,B01,60+0,Normal,"['e2e4', 'd7d5', 'b1c3', 'd5e4', 'c3e4', 'f7f5...","[' 0:01:00', ' 0:01:00', ' 0:01:00', ' 0:00:59...","[24.39086901598828, 96.03744698385567, 69.7629...","[51, -84, -97, -50, -5, -120, 63, -131, 18, No...",Bullet,47.765745,120.689820,-72.924075


In [57]:

df_cleaned.shape


(9956, 19)

In [58]:
# Saving csv file
# output_file_path = 
# Save the DataFrame as a CSV file
# df_cleaned.to_csv(output_file_path, index=False)